In [133]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import os
from bs4 import BeautifulSoup

import glob


import datetime


# options = Options()
# options.add_argument("--start-maximized")  # Maximize the browser window
# options.add_argument("--disable-extensions")
# options.add_argument("--disable-popup-blocking")
# options.add_argument("--disable-infobars")
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")
# options.add_argument("--disable-gpu")
# options.add_argument("--ignore-certificate-errors")

#driver = webdriver.Chrome()
#driver = webdriver.Safari()

ERROR = False

cookies_ec_dict = {'google': EC.element_to_be_clickable((By.XPATH, "//div[@class='QS5gu sy4vM']")),
                'nasdaq': EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'investing.com':EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")),
                'zacks.com_readmore': EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'Button__StyledButton') and contains(text(), 'Read more to accept preferences')]")),
                'zacks.com': EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'Button__StyledButton-a1qza5-0') and contains(text(), 'Accept all')]"))}

nav_nasdaq_ec_dict = {'select_date': EC.element_to_be_clickable((By.ID, "date-picker__toggle")),
                            #"//button[@class='date-picker__toggle' and @aria-label='click show datepicker popup']")),      
                      'select_searchbox':  EC.visibility_of_element_located((By.CLASS_NAME, "date-picker__input")),
                      'apply_date_button': EC.element_to_be_clickable((By.XPATH,
                            "//button[@class='date-picker__apply']"))}

nav_investingcom_ec_dict = {'select_calendar': EC.element_to_be_clickable((By.ID, "datePickerToggleBtn"))}

def accept_cookies(site = 'google'):   
    if site == 'zacks.com':
        try:
            read_more = WebDriverWait(driver, 1).until(cookies_ec_dict['zacks.com_readmore']) 
            read_more.click()
        except:
            print("No 'Read more' button found on ", site)
    try:
        accept = WebDriverWait(driver, 1).until(cookies_ec_dict[site])
        accept.click()
    except:
        print("No 'Accept' button found or unable to click on ", site)


def google(query, cookies):
    # Open the website
    driver.get("https://google.com")
    if cookies: accept_cookies('google')
    search_box = driver.find_element(By.NAME, "q")

     # Enter your search query
    search_query = query
    search_box.send_keys(search_query)

    # Simulate hitting Enter
    search_box.send_keys(Keys.RETURN)

def click_google_site(site):
    driver.find_element(By.XPATH, f"//a[contains(@href, '{site}')]").click()

def select_date(year, month, day, site = 'nasdaq'):
    if site == 'nasdaq':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['select_date']).click()
        driver.execute_script(f"document.querySelector('.date-picker__input').value = '{date}';")
        WebDriverWait(driver, 1).until(nav_nasdaq_ec_dict['e_button']).click()
    elif site == 'investing.com':
        date = datetime.date(year, month, day).strftime('%m/%d/%Y')
        WebDriverWait(driver, 1).until(nav_investingcom_ec_dict['select_calendar']).click()
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#startDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        input_element = driver.find_element(By.CSS_SELECTOR, "input.newInput#endDate")
        input_element.clear()  # Clear any existing value
        input_element.send_keys(date)  
        driver.find_element(By.ID, 'applyBtn').click()
            
def google_site(query, site, cookies):
    google(query, cookies)
    click_google_site(site)


def get_symbols(site, openclose = False):
    symbols = []
    if openclose: timings = []
    if site == 'investing.com':
        openclose_dict = {'1': 'open', '2': 'none', '3': 'close'}
        table = driver.find_element(By.ID, "earningsCalendarData")
        tbody = table.find_element(By.TAG_NAME, "tbody")
        rows = tbody.find_elements(By.TAG_NAME, "tr")
        # Iterate over each table row
        for index, row in enumerate(rows):
            if index == 0: continue
            try:
                inner_html = row.get_attribute("innerHTML")
                soup = BeautifulSoup(inner_html, "html.parser")
                print(soup)
                a_element = soup.find("a", class_="bold middle")
                if openclose:
                    timing_element = soup.find("td", class_="right time")
                    timing_value = timing_element.get("data-value")
                    timings.append(openclose_dict[timing_value])
                if a_element:
                    company = a_element.text.strip()
                    symbols.append(company)
            except Exception as e:
                print(f"Error processing row {index}: {e}")
                continue
    if openclose: return symbols, timings
    return symbols

def search_on(website, query):
    if website == 'zacks.com':
        search_input = WebDriverWait(driver, 20).until(
            EC.visibility_of_element_located((By.ID, "search-q"))
        )
        # Input the search query
        search_input.send_keys(query)
        # Press ENTER to submit the form
        search_input.send_keys(Keys.ENTER)


def get_rank(website):
    if website == 'zacks.com':
        try:
            # Wait for the paragraph element to be visible
            rank_paragraph = WebDriverWait(driver, 20).until(
                EC.visibility_of_element_located((By.CLASS_NAME, "rank_view"))
            )

            # Get the text of the paragraph element
            rank_text = rank_paragraph.text.strip()
            return int(rank_text[0])
        
        except Exception as e:
            ERROR = True

            
def analyst(website, symbol, cookies):
    if website == 'zacks.com':
        driver.get(f'https://www.zacks.com/stock/quote/{symbol}?q={symbol}')
    else:
        google_site(f'{website[:website.find(".")]} {symbol}', website, cookies)
    if cookies: accept_cookies(website)
    rank = get_rank(website)
    return rank
   
def earnings_rank(website, year, month, day, path = ''):
    if path != '':
        if path[-1] == '/': path = path[:-1]
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:   
        print('path does not exist')
    else:
        for index, tickerpath in enumerate(glob.glob(path+'/'+f'{year}-{month}-{day}/*/')):
            ticker = tickerpath[len(path+'/'+f'{year}-{month}-{day}/'):-1]
            rank = analyst(website, ticker, cookies = not(index))
            print(ticker, rank)
            with open(tickerpath+website, 'a') as f:
                f.write(f'{datetime.datetime.today().year}-{datetime.datetime.today().month}-{datetime.datetime.today().day},    {rank}\n')


def get_tickers_for_earnings_date(year, month, day, openclose = False):
    google_site('earnings calendar', 'investing.com', cookies = True)
    accept_cookies('investing.com')
    select_date(year, month, day, site = 'investing.com')
    if openclose: 
        tickers, timings = get_symbols('investing.com', openclose = openclose)
        return tickers, timings
    else:
        tickers = get_symbols('investing.com', openclose = openclose)
        return tickers
 

def make_earnings_dates_directories(path, year, month, day, tickers, openclose  = None):
    if path[-1] == '/': path = path[:-1]
    if os.path.exists(path) is False:
        os.mkdir(path)
    if os.path.exists(path+'/'+f'{year}-{month}-{day}') is False:
        os.mkdir(path+'/'+f'{year}-{month}-{day}')  
    for index, ticker in enumerate(tickers):
        if os.path.exists(path+'/'+f'{year}-{month}-{day}/'+ ticker) is False:
            os.mkdir(path+'/'+f'{year}-{month}-{day}/'+ ticker)
        if openclose is not None:
            with open(path+'/'+f'{year}-{month}-{day}/'+ ticker + '/openclose.dat', 'a') as f:
                f.write(openclose[index])

    
         


##########################################


In [134]:
driver = webdriver.Chrome()
year, month, day = 2024, 3, 7
tickers, timings = get_tickers_for_earnings_date(year, month, day, openclose = True)
# driver.quit()


<td class="flag">
<span class="ceFlags USA middle" title="United States"></span>
</td>
<td _p_pid="1131557" _r_pid="1131557" class="left noWrap earnCalCompany" title="CrowdStrike Holdings Inc">
<span class="earnCalCompanyName middle">CrowdStrike Holdings</span> (<a class="bold middle" href="/equities/crowdstrike-holdings-inc-earnings" target="_blank">CRWD</a>)
</td>
<td class="pid-1131557-2024-03-05-012024-eps_actual">--</td>
<td class="leftStrong">/  0.8233</td>
<td class="pid-1131557-2024-03-05-012024-rev_actual">--</td>
<td class="leftStrong">/  839.11M</td>
<td class="right">75.36B</td>
<td class="right time" data-value="3"><span class="marketClosed genToolTip oneliner reverseToolTip" data-tooltip="After market close"></span></td>
<td class="alert js-injected-user-alert-container" data-pair-id="1131557"><span class="js-plus-icon alertBellGrayPlus genToolTip oneliner" data-reg_ep="add alert" data-tooltip="Create Alert" data-tooltip-alt="Alert is active"></span></td>


<td class="fl

In [132]:
for tic, tim in zip(tickers, timings):
    print(tic, tim)

CRWD close
TGT open
ROST close
FERG open
PPERY none
JD open
BAYRY open
ASHTY open
FNV close
NIO open
YPF none
RADLY close
AS open
TRNO none
NDBKY none
HCXLF open
HCP close
FNLPF open
GGAL none
BOX close
NVEI close
JWN close
SBSW open
OLK none
GBTG open
ODD close
DYN none
ROVR none
BBAR none
LU none
FWRG open
MLNK close
CDRE close
BASE close
AGTI close
VVX open
SRRK none
CRCT close
TYRA none
HIBB none
ALLO none
CHPT close
RSKD open
AAGRY none
OLMA none
SEAT open
CECO open
LBPH none
VTOL close
VNTRF none
CIFR open
LRMR none
MYE open
ANAB none


In [124]:
make_earnings_dates_directories('earnings', year, month, day, tickers, openclose = timings)


In [126]:
driver = webdriver.Chrome()
earnings_rank('zacks.com', year, month, day, path = 'earnings')
driver.quit()

No 'Read more' button found on  zacks.com
USDP 3
JETMF None
AEYE 3
TLPFY None
VSCO 3
VET 4
TRIN 2
VSEC 3
ANF 2
PCT 4
NEXN 3
ARNGF None
XOMA 3
KGS 3
IMRSQ None
BFb None
CULP 4
SLNG None
CYN 3
RGLS 3
UONE None
LCSHF None
IPI 3
CNSWF None
CMTL 5
DWAY None
CURLF 2
KEQU None
MG None
MDXH 3
IOCJY None
PRVU None
RSI 3
GPRK 3
XERS 2
UONEK None
MACK None
TTNDY 3
HBB None
ADVM 3
FL 3
STOK 2
KFY 2
MLR None
SYIEY 4
HPK 5
EARN 3
EWCZ 2
REVG 3
CMAX 4
EHAB 3
VMD None
FSM 3
THM None
PAM 5
DBVT 3
MNDO None
VLGEA None
YEXT 3
REPX 3
EBS 3
BRY 4
WVE 3
YSG 3
UNFI 3
VBNK 3
MYNZ 3
EGY 5
LGGNY 1
TEO 3
GWRS 3
FRPH None
SCRYY 3
SPMYY 3
TLF None
LOMA 5
WSR 3
LVLU 3
CDVIQ None
SILV 3
BZUN None
CNVVY None
AMPY 5
INFN 3
NC None
DSGX 3
BFa None
CPB 3
FOA None
FPAY 3
EVOK None
HG 4
NGMS 3
RCMT 4
CMPO 3
TCRX 3
CC 5
MRMD 3
TPVG 3
TARA 3
CGGYY None
BRCC 3
EVGO 1
AEBZY None
ARQ 3
ADYX None
SID 3
NSPR 3
THO 4
KIDS 3
GENI 3
OSPN 1
SUPV 1
LSF 3
SOHO 3
HNST 2
CDXC 3
LYRA 2
OTCM 3
HDSN 3
BWAY 3
KMDA None
AIRG 4
DALN None
SGHC

In [2]:
import yfinance as yf

In [112]:
ticker = yf.Ticker('DELL')

In [116]:
hist = ticker.history()
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-02-02 00:00:00-05:00,85.629997,86.900002,85.220001,86.320000,3683600,0.0,0.0
2024-02-05 00:00:00-05:00,86.529999,86.870003,84.760002,85.709999,2204200,0.0,0.0
2024-02-06 00:00:00-05:00,85.730003,85.989998,82.160004,82.680000,5013800,0.0,0.0
2024-02-07 00:00:00-05:00,83.440002,83.830002,82.779999,83.739998,2964200,0.0,0.0
2024-02-08 00:00:00-05:00,83.610001,85.139999,83.510002,84.800003,2993800,0.0,0.0
2024-02-09 00:00:00-05:00,85.250000,86.650002,85.010002,86.199997,2315700,0.0,0.0
2024-02-12 00:00:00-05:00,86.500000,86.889999,85.830002,86.620003,2979100,0.0,0.0
2024-02-13 00:00:00-05:00,84.989998,85.190002,83.449997,83.980003,3144300,0.0,0.0
2024-02-14 00:00:00-05:00,85.059998,86.230003,84.720001,86.040001,2361300,0.0,0.0


In [115]:
hist

<bound method log_indent_decorator.<locals>.wrapper of yfinance.Ticker object <DELL>>

In [94]:
hist['Low']['2024-02-02']

85.22000122070312

In [10]:
ticker = yf.download(tickers="DELL", period="5d", interval="1m", prepost = True)

[*********************100%%**********************]  1 of 1 completed


In [11]:
# start = datetime.datetime(2021, 10, 18, 9, 30, 0)
# end = datetime.datetime(2021, 10, 18, 10, 30, 0)

# filtered = ticker[start: end]
# filtered

ticker['2024-03-01']

/var/folders/9g/bzb1c3hn6jvdx57km19ky1y00000gn/T/ipykernel_11591/453447690.py:7: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  ticker['2024-03-01']


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-03-01 04:00:00-05:00,114.000,114.93,113.0000,114.11,114.11,0
2024-03-01 04:01:00-05:00,114.480,114.82,114.4700,114.82,114.82,0
2024-03-01 04:02:00-05:00,114.800,115.00,114.5700,114.60,114.60,0
2024-03-01 04:03:00-05:00,114.600,114.79,114.1100,114.79,114.79,0
2024-03-01 04:04:00-05:00,114.770,115.00,114.7400,114.91,114.91,0
...,...,...,...,...,...,...
2024-03-01 19:55:00-05:00,124.190,124.20,124.1492,124.15,124.15,0
2024-03-01 19:56:00-05:00,124.150,124.25,124.1100,124.19,124.19,0
2024-03-01 19:57:00-05:00,124.200,124.20,124.0000,124.00,124.00,0
